In [6]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from DeepTCN.DeepTCN import DeepTCN
from DeepTCN.plots import plot_matplotlib
from data_process import get_data_loaders, getFeatures
from tensorflow.keras.models import load_model
# updated notebook

In [7]:
data =pd.read_csv("flights.csv")
input_seq_len = 10
output_seq_len = 2
test_size = 0.2
val_size = 0.25
batch_size = 64
rand_state = 42

data = get_data_loaders(data, input_seq_len, output_seq_len, test_size, val_size, batch_size, rand_state)[0]
features = getFeatures()
y = data['energy_consumed'].values
x = data[features].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
y = y.reshape(-1, 1)

In [8]:
print("x shape: ", x.shape)
print("y shape: ", y.shape)

x shape:  (257896, 32)
y shape:  (257896, 1)


In [9]:
# Define the parameters for the DeepTCN model
forecast_period = 2  # You need to specify the forecast period you want to predict
lookback_period = 10  # You need to specify the number of past observations the model should look at
quantiles=[0.1, 0.5, 0.9]  # Specify the quantiles you are interested in predicting
filters = 32  # The number of filters in the TCN layers
kernel_size = 2  # The kernel size for the convolutional layers
dilation_rates = [1, 2, 4, 8]  # The dilation rates for the TCN layers
units = 64  # The number of units in the dense layer
loss = 'nonparametric'  # Specify the loss function to be used

# Initialize the DeepTCN model
model = DeepTCN(
    y=y,
    x=x,
    forecast_period=forecast_period,
    lookback_period=lookback_period,
    quantiles=quantiles,
    filters=filters,
    kernel_size=kernel_size,
    dilation_rates=dilation_rates,
    units=units,
    loss=loss
)

In [10]:
# Fit the model 
model.fit(
    learning_rate=0.001,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    verbose=1
)

model.save_model()

Epoch 1/10
6448/6448 [==============================] - 42s 6ms/step - loss: 0.0367 - val_loss: 0.0136
Epoch 2/10
6448/6448 [==============================] - 38s 6ms/step - loss: 0.0101 - val_loss: 0.0081
Epoch 3/10
6448/6448 [==============================] - 38s 6ms/step - loss: 0.0071 - val_loss: 0.0057
Epoch 4/10
6448/6448 [==============================] - 39s 6ms/step - loss: 0.0056 - val_loss: 0.0069
Epoch 5/10
6448/6448 [==============================] - 63s 10ms/step - loss: 0.0047 - val_loss: 0.0046
Epoch 6/10
6448/6448 [==============================] - 38s 6ms/step - loss: 0.0042 - val_loss: 0.0041
Epoch 7/10
6448/6448 [==============================] - 37s 6ms/step - loss: 0.0039 - val_loss: 0.0034
Epoch 8/10
6448/6448 [==============================] - 38s 6ms/step - loss: 0.0036 - val_loss: 0.0098
Epoch 9/10
6448/6448 [==============================] - 38s 6ms/step - loss: 0.0034 - val_loss: 0.0044
Epoch 10/10
6448/6448 [==============================] - 39s 6ms/step - 

In [11]:
"""
y: np.array.
            Past values of target time series, array with shape (n_samples, n_targets) where n_samples is the length
            of the time series and n_targets is the number of target time series. The number of past samples provided
            (n_samples) should not be less than the length of the lookback period.
            
x: np.array.
    Past and future values of features time series, array with shape (n_samples + n_forecast, n_features) where
    n_samples is the length of the time series, n_forecast is the decoder length and n_features is the number
    of features time series. The number of past samples provided (n_samples) should not be less than the length
    of the lookback period.
"""

# Predict the quantiles for the test set
data

,flight,time,wind_speed,wind_angle,battery_voltage,battery_current,position_x,position_y,position_z,orientation_x,...,route,max_altitude,min_altitude,mean_altitude,power,time_diff,current_atm,energy_atm,current_consumed,energy_consumed
0,1,0.00,0.005525,0.033426,0.763874,0.008701,0.002053,0.999960,0.715950,0.570210,...,0.4,0.25,0.25,0.25,0.010330,0.000000,0.002039,0.002267,0.003602,0.004199
1,1,0.20,0.005525,0.008357,0.764581,0.008869,0.002053,0.999960,0.715949,0.570205,...,0.4,0.25,0.25,0.25,0.010521,0.037879,0.002185,0.002432,0.003605,0.004202
2,1,0.30,0.005525,0.980501,0.764404,0.008869,0.002053,0.999960,0.715952,0.570205,...,0.4,0.25,0.25,0.25,0.010521,0.018939,0.002112,0.002349,0.003607,0.004204
3,1,0.50,0.005525,0.986072,0.764228,0.008869,0.002053,0.999960,0.715956,0.570213,...,0.4,0.25,0.25,0.25,0.010521,0.037879,0.002185,0.002432,0.003610,0.004207
4,1,0.60,0.005525,1.000000,0.762284,0.008534,0.002053,0.999960,0.715960,0.570213,...,0.4,0.25,0.25,0.25,0.010139,0.018939,0.002100,0.002336,0.003611,0.004209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257891,279,152.10,0.060773,0.551532,0.571226,0.008869,0.002048,0.999984,0.721872,0.590608,...,0.6,1.00,0.25,0.50,0.010392,0.018939,0.002112,0.002345,0.422761,0.411081
257892,279,152.20,0.060773,0.545961,0.569813,0.008869,0.002048,0.999984,0.721904,0.590608,...,0.6,1.00,0.25,0.50,0.010391,0.018939,0.002112,0.002345,0.422762,0.411083
257893,279,152.41,0.066298,0.526462,0.571050,0.009203,0.002048,0.999984,0.721937,0.590610,...,0.6,1.00,0.25,0.50,0.010751,0.039773,0.002218,0.002457,0.422766,0.411087
257894,279,152.60,0.060773,0.520891,0.570873,0.009538,0.002048,0.999984,0.721946,0.590619,...,0.6,1.00,0.25,0.50,0.011110,0.035985,0.002224,0.002464,0.422770,0.411091


In [14]:
# Generate some time series
N = 500
t = np.linspace(0, 1, N)
e = np.random.multivariate_normal(mean=np.zeros(3), cov=np.eye(3), size=N)
a = 10 + 10 * t + 10 * np.cos(2 * np.pi * (10 * t - 0.5)) + 1 * e[:, 0]
b = 20 + 20 * t + 20 * np.cos(2 * np.pi * (20 * t - 0.5)) + 2 * e[:, 1]
c = 30 + 30 * t + 30 * np.cos(2 * np.pi * (30 * t - 0.5)) + 3 * e[:, 2]
y = np.hstack([a.reshape(-1, 1), b.reshape(-1, 1), c.reshape(-1, 1)])

# Fit the model
model = DeepTCN(
    y=y,
    x=None,
    forecast_period=100,
    lookback_period=100,
    quantiles=[0.001, 0.1, 0.5, 0.9, 0.999],
    filters=3,
    kernel_size=3,
    dilation_rates=[1],
    loss='parametric'
)

model.fit(
    learning_rate=0.001,
    batch_size=16,
    epochs=300,
    verbose=1
)

# Generate the forecasts
df = model.forecast(y=y, x=1)
# # Generate some time series
# N = 500
# t = np.linspace(0, 1, N)
# e = np.random.multivariate_normal(mean=np.zeros(3), cov=np.eye(3), size=N)
# a = 10 + 10 * t + 10 * np.cos(2 * np.pi * (10 * t - 0.5)) + 1 * e[:, 0]
# b = 20 + 20 * t + 20 * np.cos(2 * np.pi * (20 * t - 0.5)) + 2 * e[:, 1]
# c = 30 + 30 * t + 30 * np.cos(2 * np.pi * (30 * t - 0.5)) + 3 * e[:, 2]
# y = np.hstack([a.reshape(-1, 1), b.reshape(-1, 1), c.reshape(-1, 1)])

# # Fit the model
# model = DeepTCN(
#     y=y,
#     x=None,
#     forecast_period=100,
#     lookback_period=100,
#     quantiles=[0.001, 0.1, 0.5, 0.9, 0.999],
#     filters=3,
#     kernel_size=3,
#     dilation_rates=[1],
#     loss='parametric'
# )

# model.fit(
#     learning_rate=0.001,
#     batch_size=16,
#     epochs=300,
#     verbose=1
# )

# # Generate the forecasts
# df = model.forecast(y=y, x=None)

# print(y.shape)

# # Plot the forecasts
# fig = plot_matplotlib(df=df, quantiles=[0.001, 0.1, 0.5, 0.9, 0.999], x_axis_label="time", y_axis_label="value")
# fig.write_image('results.png', scale=4, height=900, width=700)


Epoch 1/300
19/19 [==============================] - 1s 2ms/step - loss: 3.7076
Epoch 2/300
19/19 [==============================] - 0s 2ms/step - loss: 2.2471
Epoch 3/300
19/19 [==============================] - 0s 2ms/step - loss: 1.6934
Epoch 4/300
19/19 [==============================] - 0s 2ms/step - loss: 1.4005
Epoch 5/300
19/19 [==============================] - 0s 2ms/step - loss: 1.1974
Epoch 6/300
19/19 [==============================] - 0s 2ms/step - loss: 1.0376
Epoch 7/300
19/19 [==============================] - 0s 2ms/step - loss: 0.8977
Epoch 8/300
19/19 [==============================] - 0s 2ms/step - loss: 0.7661
Epoch 9/300
19/19 [==============================] - 0s 2ms/step - loss: 0.6397
Epoch 10/300
19/19 [==============================] - 0s 2ms/step - loss: 0.5216
Epoch 11/300
19/19 [==============================] - 0s 2ms/step - loss: 0.4080
Epoch 12/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2986
Epoch 13/300
19/19 [=================

AttributeError: 'DeepTCN' object has no attribute 'x_min'